# U-Net by Lukas

In [1]:
import tensorflow as tf
import os
import random
import numpy as np
 
from tqdm import tqdm 

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras import layers, metrics


from tensorboard.plugins.hparams import api as hp

## 1. Experiment setup and the HParams experiment summary

Experiment with three hyperparameters in the model:

1. Number of channels (1x or 2x)
2. Learning rate
3. Batch size
4. Epochs
5. Picture size
6. (Optimizer)

In [ ]:
HP_CHANNELS = hp.HParam('channels', hp.Discrete([1, 2]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.0001, 0.01))  
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32, 64]))
HP_EPOCHS = hp.HParam('epochs', hp.Discrete([10, 20]))
HP_IMAGE_SIZE = hp.HParam('image_size', hp.Discrete([1, 2]))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_F1SCORE = 'f1_score'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_CHANNELS, HP_LEARNING_RATE, HP_BATCH_SIZE, HP_EPOCHS, HP_IMAGE_SIZE],
    metrics=[hp.Metric(METRIC_F1SCORE, display_name='F1-score')],
  )

In [ ]:
def display(display_list):
  plt.figure(figsize=(8, 8))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    if i == 1:
      plt.imshow(display_list[i], cmap='gray',  interpolation='nearest')
    elif i == 2:
      plt.imshow(display_list[i], cmap='jet',  interpolation='nearest')
    else:
      plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
def train_test_model(hparams):
    #image dimensions, seems to work with non-square inputs
    IMAGE_CHANNELS = 3

    IMAGE_HEIGHT =  192*hparams[HP_IMAGE_SIZE]
    IMAGE_WIDTH = 64*hparams[HP_IMAGE_SIZE]

    seed = 4
    np.random.seed = seed
    random.seed(seed)
    tf.random.set_seed(seed)

    DATA_TRAIN = "./datasets/KolektorSDD2/train/"
    DATA_TEST = "./datasets/KolektorSDD2/test/"



    train_ids = next(os.walk(os.path.join(DATA_TRAIN, "images/")))[2]
    test_ids = next(os.walk(os.path.join(DATA_TEST, "images/")))[2]


    X_train = np.zeros((len(train_ids), IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS), dtype=np.float16)
    y_train = np.zeros((len(train_ids), IMAGE_HEIGHT, IMAGE_WIDTH), dtype=np.float16)

    print('Resizing training images and masks')

    for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):   
        path = DATA_TRAIN 

        img = imread(path + 'images/' + id_)[:,:,:IMAGE_CHANNELS]  
        img = resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', preserve_range=True)
        img /= 255.0
        X_train[n] = img  #Fill empty X_train with values from img

        mask = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 1), dtype=bool)
        mask_file = os.path.join(path + 'masks/' + id_[:5] + "_GT.png")
        mask = imread(mask_file)[:,:]

        mask = resize(mask, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', preserve_range=True)
        mask /= 255.0  
        mask = np.where(mask > 0.5, 1.0, 0.0) 
        y_train[n] = mask 
        
    # test images
    test_images = np.zeros((len(test_ids), IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS), dtype=np.float16)
    test_masks = np.zeros((len(test_ids), IMAGE_HEIGHT, IMAGE_WIDTH), dtype=np.float16)

    sizes_test = []
    print('Resizing test images') 
    for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
        path = DATA_TEST
        img = imread(path + '/images/' + id_ )[:,:,:IMAGE_CHANNELS]
        sizes_test.append([img.shape[0], img.shape[1]])
        img = resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', preserve_range=True)
        img /= 255.0
        test_images[n] = img

        mask = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 1), dtype=bool)
        mask_file = os.path.join(path + 'masks/' + id_[:5] + "_GT.png")
        mask = imread(mask_file)[:,:]

        mask = resize(mask, (IMAGE_HEIGHT, IMAGE_WIDTH), mode='constant', preserve_range=True)
        mask /= 255.0     
        mask = np.where(mask > 0.5, 1.0, 0.0)   
        test_masks[n] = mask 

    print('Dataset is ready')

    non_zero = np.count_nonzero(y_train)
    print(non_zero)
    # print(non_zero/(IMAGE_HEIGHT*IMAGE_WIDTH*len(y_train))*100)

    unique, counts = np.unique(y_train, return_counts=True)

    print(dict(zip(unique, counts)))
    print("Percentage of faulty images in train data:", counts[1]/(counts[0]+counts[1])*100, " %")
    neg = counts[0]
    pos = counts[1]
   
    initial_bias = np.log([pos/neg])
    output_bias = tf.keras.initializers.Constant(initial_bias)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state = seed)
    

    #Building U-net model
    #Downward stream
    inputs = tf.keras.layers.Input((IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS))
    conv_11 = layers.Conv2D(16*hparams[HP_CHANNELS],kernel_size=(3,3), activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(inputs)
    conv_12 = layers.Conv2D(16*hparams[HP_CHANNELS],kernel_size=(3,3), activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_11)#TODO: Understand parameters

    max_pool_1 = layers.MaxPool2D((2,2))(conv_12)
    conv_21 = layers.Conv2D(32*hparams[HP_CHANNELS],(3,3),activation = 'relu',padding= 'same',kernel_initializer = 'he_normal')(max_pool_1)
    conv_22 = layers.Conv2D(32*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_21)

    max_pool_2 = layers.MaxPool2D((2,2))(conv_22)
    conv_31 = layers.Conv2D(64*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(max_pool_2)
    conv_32 = layers.Conv2D(64*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_31)

    max_pool_3 = layers.MaxPool2D((2,2))(conv_32)
    conv_41 = layers.Conv2D(128*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(max_pool_3)
    conv_42 = layers.Conv2D(128*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding= 'same')(conv_41)

    max_pool_4 = layers.MaxPool2D((2,2))(conv_42)
    conv_51 = layers.Conv2D(256*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(max_pool_4)
    conv_52 = layers.Conv2D(256*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_51)

    #Upward stream
    upconv_1 = layers.Conv2DTranspose(128*hparams[HP_CHANNELS],(2,2), strides=(2,2))(conv_52)
    upconv_1_conc = layers.concatenate([upconv_1,conv_42])
    conv_61 = layers.Conv2D(128*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_1_conc)
    conv_62 = layers.Conv2D(128*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_61)

    upconv_2 = layers.Conv2DTranspose(64*hparams[HP_CHANNELS], (2,2), strides = (2,2))(conv_62)
    upconv_2_conc = layers.concatenate([upconv_2, conv_32])
    conv_71 = layers.Conv2D(64*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_2_conc)
    conv_72 = layers.Conv2D(64*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_71)

    upconv_3 = layers.Conv2DTranspose(32*hparams[HP_CHANNELS],(2,2), strides=(2,2))(conv_72)
    upconv_3_conc = layers.concatenate([upconv_3,conv_22])
    conv_81 = layers.Conv2D(32*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_3_conc)
    conv_82 = layers.Conv2D(32*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_81)

    upconv_4 = layers.Conv2DTranspose(16*hparams[HP_CHANNELS],(2,2), strides=(2,2))(conv_82)
    upconv_4_conc = layers.concatenate([upconv_4,conv_12])
    conv_91 = layers.Conv2D(16*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_4_conc)
    conv_92 = layers.Conv2D(16*hparams[HP_CHANNELS],(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_91)
    outputs = layers.Conv2D(1,(1,1), activation = 'sigmoid', padding = 'same',kernel_initializer = 'he_normal', bias_initializer=output_bias)(conv_92)#TODO: Check function here

    model = tf.keras.Model(inputs = [inputs], outputs = [outputs])

    from keras.optimizers import Adam
    optimizer = Adam(learning_rate = hparams[HP_LEARNING_RATE])

    #Compiling model
    model.compile(optimizer=optimizer , loss='binary_crossentropy', metrics=['f1_score']) #TODO: Parameters check #metrics.BinaryIoU()
    model.summary()

    from keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15, restore_best_weights=True)

    epochs = hparams[HP_EPOCHS]  
    batch_size = hparams[HP_BATCH_SIZE]

    history = model.fit(X_train, 
                        y_train, 
                        epochs = epochs, 
                        validation_data = (X_val, y_val), 
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        callbacks=[early_stopping]) 

    # Y_pred = model.predict(test_images)

    model.save(f"trainedModels/hptuning_session{session_num}.h5")

    # model.fit(x_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
    _, accuracy = model.evaluate(test_images, test_masks)
    return accuracy

In [ ]:
def run(run_dir, hparams, session_num):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams, session_num)
    tf.summary.scalar(METRIC_F1SCORE, accuracy, step=1)

In [ ]:
session_num = 0

for channels in HP_CHANNELS.domain.values:
  for image_size in HP_IMAGE_SIZE.domain.values:
    for learning_rate in HP_LEARNING_RATE.domain.values:
      for batch_size in HP_BATCH_SIZE.domain.values:
        for epochs in HP_EPOCHS.domain.values:
          hparams = {
              HP_CHANNELS: channels,
              HP_IMAGE_SIZE: image_size,
              HP_LEARNING_RATE: learning_rate,
              HP_BATCH_SIZE: batch_size,
              HP_EPOCHS: epochs,
          }
          run_name = "run-%d" % session_num
          print('--- Starting trial: %s' % run_name)
          print({h.name: hparams[h] for h in hparams})
          run('logs/hparam_tuning/' + run_name, hparams, session_num)
          session_num += 1

# HP_CHANNELS = hp.HParam('channels', hp.Discrete([1, 2]))
# HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.0001, 0.01))  
# HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32, 64]))
# HP_EPOCHS = hp.HParam('epochs', hp.Discrete([10, 20]))
# HP_IMAGE_SIZE = hp.HParam('image_size', hp.Discrete([1, 2]))

In [ ]:
%tensorboard --logdir logs/hparam_tuning